WRC+ Predections

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/')

import predections

In [26]:
nn_02_preds = pd.read_csv('./models/wrc_models/2025_Predections/nn_02_23_preds.csv')
nn_15_preds = pd.read_csv('./models/wrc_models/2025_Predections/nn_15_23_preds.csv')
nn_23_preds = pd.read_csv('./models/wrc_models/2025_Predections/nn_23_preds.csv')
reg_02_preds = pd.read_csv('./models/wrc_models/2025_Predections/reg_02_23_preds.csv')
reg_15_preds = pd.read_csv('./models/wrc_models/2025_Predections/reg_15_23_preds.csv')
reg_23_preds = pd.read_csv('./models/wrc_models/2025_Predections/reg_23_preds.csv')

basic cleaning

In [27]:
reg_02_preds = reg_02_preds.rename(columns={'Name': 't1'})
reg_15_preds = reg_15_preds.rename(columns={'Name': 't1'})
reg_23_preds = reg_23_preds.rename(columns={'Name': 't1'})

In [28]:
reg_02_preds = reg_02_preds.rename(columns={'0': 'proj_wrc+'})
reg_15_preds = reg_15_preds.rename(columns={'0': 'proj_wrc+'})
reg_23_preds = reg_23_preds.rename(columns={'0': 'proj_wrc+'})

In [29]:
nn_02_preds = nn_02_preds.drop(columns=['Unnamed: 0'])
nn_15_preds = nn_15_preds.drop(columns=['Unnamed: 0'])
nn_23_preds = nn_23_preds.drop(columns=['Unnamed: 0'])
reg_02_preds = reg_02_preds.drop(columns=['Unnamed: 0'])
reg_15_preds = reg_15_preds.drop(columns=['Unnamed: 0'])
reg_23_preds = reg_23_preds.drop(columns=['Unnamed: 0'])

uniform names

In [30]:
def clean_names(df):
    df['t1'] = df['t1'].str.split(',').str[0]
    return df

In [31]:
# apply function
nn_02_preds = clean_names(nn_02_preds)
nn_15_preds = clean_names(nn_15_preds)
nn_23_preds = clean_names(nn_23_preds)

#### AVG of Projections

merge dfs together

In [34]:
# merge all dfs together
merged_preds = nn_02_preds.rename(columns={'proj_wrc+': 'proj_wrc+_N02'})

merged_preds = merged_preds.merge(
    nn_15_preds,
    on='t1',
    how='outer',
    suffixes=('', "_N15")
)

merged_preds = merged_preds.merge(
    nn_23_preds,
    on='t1',
    how='outer',
    suffixes=('', '_N23')
)

merged_preds = merged_preds.merge(
    reg_02_preds,
    on='t1',
    how='outer',
    suffixes=("", "_R02")
)

merged_preds = merged_preds.merge(
    reg_15_preds,
    on='t1',
    how='outer',
    suffixes=('', '_R15')
)

merged_preds = merged_preds.merge(
    reg_23_preds,
    on='t1',
    how='outer',
    suffixes=('', '_R23')
)

averages
<br>
<small> Adjustement Factor applied because WRC+ is centered at 100 and the models were consitently undershooting. Long term I want to use an xgboost model to predict outcomes but I currently do not have the PA model. I also think age adjustements need to be applied first. 

In [38]:
# Calculate weighted average proj_wrc+ across all models
merged_preds['avg_proj_wrc+'] = (
    merged_preds['proj_wrc+_N23'] * .25 +
    merged_preds['proj_wrc+'] * 0.125 +
    merged_preds['proj_wrc+_N02'] * 0.125 +
    merged_preds['proj_wrc+_R23'] * .25 +
    merged_preds['proj_wrc+_R15'] * 0.125 +
    merged_preds['proj_wrc+_R02'] * 0.125
)

# Calculate the current mean
current_mean = merged_preds['avg_proj_wrc+'].mean()

# Adjust values to make mean = 100
adjustment_factor = 100 / current_mean
merged_preds['avg_proj_wrc+'] = merged_preds['avg_proj_wrc+'] * adjustment_factor

# Sort by average projection
merged_preds = merged_preds.sort_values('avg_proj_wrc+', ascending=False)